In [161]:
# tampering with the kwd file on passaro
# the /experiment folder is on the ssd of passaro (/usr/local/experiment/raw_data)

%matplotlib inline
from __future__ import division
import pdb
import sys
import pandas as pd
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os
import wave
import struct
import h5py
from scipy.signal import hilbert
from basic_viewing import h5_functions as h5
import glob
import errno    
import os
import shutil as sh

matplotlib.style.use('ggplot')


# Check wihic computer to decide where the things are mounted
comp_name=socket.gethostname()
print 'Computer: ' + comp_name
if  comp_name == 'chim':
    #sys.path.append('/Users/zeke/experiment/ephysDataManagement/ephysScripts')
    experiment_folder = os.path.join(os.path.abspath('W:') , "earneodo", "bci_zf")
elif  'lookfar' in comp_name:
    repos_folder = os.path.abspath('/Users/zeke/reposs')
    experiment_folder = os.path.join(os.path.abspath('/Volumes') , 'gentner', 'earneodo', 'bci_zf')

elif 'lintu' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')

elif 'passaro' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    cube_experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')
    store_experiment_folder = os.path.join('/Data/bci_zf/')
    experiment_folder = os.path.join('/usr/local/experiment')

sys.path.append(os.path.join(repos_folder, 'soundflow', 'sound_tools'))
sys.path.append(os.path.join(repos_folder, 'ephysflow'))
sys.path.append(os.path.join(repos_folder, 'analysis-tools'))

import soundtools as st
import Kwik as oe
from scipy.io import wavfile

raw_data_folder = os.path.join(experiment_folder, 'raw_data')
ss_data_folder = os.path.join(cube_experiment_folder, 'ss_data')

def create_neural_data_set(data_set, parent_group, channel_list, frame_size=None, processing=None, *args, **kwargs):
    # make the new dataset
    d_type = data_set.dtype
    nd_cols = np.array(channel_list).size
    nd_rows = data_set.shape[0]
    d_chunks = np.array(data_set.chunks)
    
    if d_chunks[0] > nd_rows:
        d_chunks[0] = nd_rows
    
    if frame_size is None:
        frame_size = d_chunks[0] if d_chunks[0]<nd_rows else nd_rows
    
    neural_dset = parent_group.create_dataset("data", (nd_rows, nd_cols), 
                                              chunks=(d_chunks[0], nd_cols), dtype=d_type)
    
    copy_frame = np.zeros([frame_size, nd_cols], dtype=d_type)
    n_full_frames = int(data_set.shape[0]/frame_size)
    #print neural_dset.shape
    # Fill the dataset:
    for i in range(n_full_frames):
        copy_frame = h5.load_table_slice(data_set, np.arange(i*frame_size, (i+1)*frame_size) , channel_list)
        #print copy_frame.shape
        #neural_dset.write_direct(copy_frame, dest_sel=np.s_[i*frame_size: (i+1)*frame_size, 0:nd_cols])
        neural_dset[i*frame_size: (i+1)*frame_size, 0:nd_cols] = copy_frame
    
    if frame_size*n_full_frames < nd_rows:
        last_frame_size = nd_rows - frame_size*n_full_frames
        copy_frame = np.zeros([last_frame_size, nd_cols], dtype=d_type)
        copy_frame = h5.load_table_slice(data_set, 
                                         np.arange(n_full_frames*frame_size, 
                                                   n_full_frames*frame_size + last_frame_size),
                                         channel_list)
    # copy the attributes:
    neural_dset.attrs.create('valid_samples', np.array(data_set.attrs['valid_samples'][channel_list]))
    
def copy_attribs(source, dest):
    for key, attrib in source.attrs.iteritems():
        dest.attrs.create(key, attrib, dtype=attrib.dtype)
        
def copy_application_data(source_rec, dest_rec, chan_list,
                          resize_keys = ['channel_bit_volts', 'channel_sample_rates']):
    
    dest_rec.create_group('application_data')
    for key, attrib in source_rec['application_data'].attrs.iteritems():
        if not key in resize_keys:
            dest_rec['application_data'].attrs.create(key, attrib, dtype=attrib.dtype)
        else:
            #print key
            #print attrib[chan_list]
            dest_rec['application_data'].attrs.create(key, attrib[chan_list], dtype=attrib.dtype)

def check_rec_data(source_rec):
    #check the data table of a rec group
    data_set = source_rec['data']
    valid_samples = None
    try:
        valid_samples = data_set.attrs.get('valid_samples')
    except:
        pass
    
    has_data = True if valid_samples is not None and valid_samples[0]>0 else False
    return has_data

def get_valid_samples(source_rec):
    #get the valid samples declared by the data table attributes
    data_set = source_rec['data']
    valid_samples = None
    try:
        valid_samples = data_set.attrs.get('valid_samples')[0]
    except:
        pass
    return valid_samples

def create_data_groups(raw_file, new_file, chan_list):
    for rec_name, rec_group in raw_file['/recordings'].iteritems():
        new_file['/recordings'].create_group(rec_name)
        copy_attribs(rec_group, new_file['/recordings'][rec_name])
        copy_application_data(rec_group, new_file['/recordings'][rec_name], chan_list)
        create_neural_data_set(rec_group['data'], new_file['/recordings'][rec_name], chan_list)
        
def make_shank_kwd(raw_file, out_file_path, chan_list):
    ss_file = h5py.File(out_file_path, 'w')
    copy_attribs(raw_file, ss_file)
    ss_file.create_group('/recordings')
    create_data_groups(raw_file, ss_file, chan_list) 
    ss_file.close()
    
def list_flatten(x):
    return [val for sublist in x for val in sublist]

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def make_super_file(path):
    new_file = h5py.File(path, 'w')
    new_file.create_group('/recordings')
    new_file.close()

def create_data_group(raw_file, new_file, chan_list, new_rec_name):
    for rec_name, rec_group in raw_file['/recordings'].iteritems():
        new_file['/recordings'].create_group(rec_name)
        copy_attribs(rec_group, new_file['/recordings'][rec_name])
        copy_application_data(rec_group, new_file['/recordings'][rec_name], chan_list)
        create_neural_data_set(rec_group['data'], new_file['/recordings'][rec_name], chan_list)

def insert_neural_rec_group(dest_file, raw_rec_group, chan_list, new_group_name=None):
    rec_name = raw_rec_group.name if new_group_name is None else new_group_name
    
    dest_file['/recordings'].create_group(rec_name)
    new_rec = dest_file['/recordings'][rec_name]
    copy_application_data(raw_rec_group, new_rec, chan_list)
    copy_attribs(raw_rec_group, new_rec)
    create_neural_data_set(raw_rec_group['data'], new_rec, chan_list)
    
def modify_rec_group_attribs(kwd_file, rec_name, attr_dict, new_attr_dict = None):
    for key, value in attr_dict.iteritems():
        kwd_file['/recordings'][rec_name].attrs.modify(key, value)
    
    if new_attr_dict is not None:
        for key, value in new_attr_dict.iteritems():
            kwd_file['/recordings'][rec_name].attrs.create(key, value)

def get_recs_list(experiment_file):
    if experiment_file['/recordings'].keys() == []:
        rec_list = None
    else:
        rec_list = np.array([int(i) for i in experiment_file['/recordings'].keys()])
    return rec_list

def get_experiment_endpoints(experiment_file):
    recs_list = get_recs_list(experiment_file)
    if recs_list is not None:
        last_rec_num = np.max(recs_list)
        last_rec_name = str(last_rec_num)
        last_rec = experiment_file['/recordings'][last_rec_name]
        last_rec_start = last_rec.attrs['start_sample']
        last_rec_nsamples = last_rec['data'].shape[0]
        next_rec_num = last_rec_num + 1
        next_sample = last_rec_start + last_rec_nsamples
    else:
        next_rec_num = 0
        next_sample = 0
        
    return [next_rec_num, next_sample]
            

def insert_experiment_groups(dest_file, raw_file, chan_list):
    # all the recs in an experiment file if they pass check
    for raw_rec_name, raw_rec_group in raw_file['/recordings'].iteritems():
        #print 'rec {0}'.format(raw_rec_name)
        s_f = raw_rec_group.attrs['sample_rate']
        if check_rec_data(raw_rec_group):
            target_rec_num, target_start_sample = get_experiment_endpoints(dest_file)
            target_rec_name = str(target_rec_num)
            target_start_time = int(target_start_sample/(0.001*s_f))
            target_source = '{0}:''/''recordings/{1}'.format(raw_file.filename, raw_rec_name)
            insert_neural_rec_group(dest_file, raw_rec_group, chan_list, new_group_name=target_rec_name)
            modify_rec_group_attribs(dest_file, target_rec_name, 
                             {'start_sample' : target_start_sample},
                              new_attr_dict = {'name' : target_source,
                                               'start_time' : target_start_time})
        else:
            #print "Skipping rec {0} with no data".format(raw_rec_name)
            pass

Computer: passaro


In [171]:
bird_id = 'z017'
sess_day = '2016-06-20'
depth = '150'

channel_config = {'neural': [0, 11, 9, 16, 14, 5, 8, 10, 15, 2, 7, 4, 3, 5, 1, 16, 15],
                  'super' : range(17),
                  '0' : [11, 15, 8, 10],
                  '1' : [14, 9, 13, 12],
                  '2' : [2, 3, 1, 6],
                  '3' : [5, 7, 0, 4], 
                  'test' : [11],
                 'sound': [17]}

chan_list= channel_config['super']
# Make the super session and gather the list of sessions
# get the sessions list of files
sessions = glob.glob(os.path.join(raw_data_folder, bird_id, sess_day + '*'))
experiments = list_flatten([ glob.glob(os.path.join(s, '*.raw.kwd'))[:] for s in sessions ])
experiments.sort()

In [181]:
bird_id = 'z017'
sess_day = '2016-06-20'
depth = 225

channel_config = {'neural': [0, 11, 9, 16, 14, 5, 8, 10, 15, 2, 7, 4, 3, 5, 1, 16, 15],
                  'super' : range(17),
                  '0' : [11, 15, 8, 10],
                  '1' : [14, 9, 13, 12],
                  '2' : [2, 3, 1, 6],
                  '3' : [5, 7, 0, 4], 
                  'test' : [11],
                 'sound': [17]}

chan_list= channel_config['super']
# Make the super session and gather the list of sessions
# get the sessions list of files
sessions = glob.glob(os.path.join(raw_data_folder, bird_id, sess_day + '*' + str(depth)))
experiments = list_flatten([ glob.glob(os.path.join(s, '*.raw.kwd'))[:] for s in sessions ])
experiments.sort()

In [182]:
sessions = glob.glob(os.path.join(raw_data_folder, bird_id, sess_day + '*' + str(depth)))
print sessions

['/usr/local/experiment/raw_data/z017/2016-06-20_08-55-34_225']


In [196]:
bird_id = 'z017'
sess_day = '2016-07-07'
depth = 225

channel_config = {'neural': [0, 11, 9, 16, 14, 5, 8, 10, 15, 2, 7, 4, 3, 5, 1, 16, 15],
                  'super' : range(17),
                  '0' : [11, 15, 8, 10],
                  '1' : [14, 9, 13, 12],
                  '2' : [2, 3, 1, 6],
                  '3' : [5, 7, 0, 4], 
                  'test' : [11],
                 'sound': [17]}

chan_list= channel_config['super']
# Make the super session and gather the list of sessions
# get the sessions list of files
sessions = glob.glob(os.path.join(raw_data_folder, bird_id, sess_day + '*' + str(depth)))
experiments = list_flatten([ glob.glob(os.path.join(s, '*.raw.kwd'))[:] for s in sessions ])
experiments.sort()

# make the supersession file
super_sess_path = os.path.join(ss_data_folder, bird_id, 'day-' + sess_day)
mkdir_p(super_sess_path)
super_file_path = os.path.join(super_sess_path, 'experiment.raw.kwd')
print super_file_path
make_super_file(super_file_path)

super_file = h5py.File(super_file_path, 'a')
for experiment_path in experiments:
    print 'File {0}'.format(experiment_path)
    raw_file = h5py.File(experiment_path, 'r')
    insert_experiment_groups(super_file, raw_file, chan_list)
    raw_file.close()
    super_file.flush()

super_file.close()

#make the backups
bkp_dest_path = os.path.join(cube_experiment_folder, 'raw_data', bird_id)
store_dest_path = os.path.join(store_experiment_folder, 'raw_data', bird_id)

mkdir_p(bkp_dest_path)
mkdir_p(store_dest_path)
for session_path in sessions:
    session_name = os.path.split(session_path)[-1]
    print "cube backup: " + session_name
    dest_bkp = os.path.join(bkp_dest_path, session_name)
    try:
        sh.copytree(session_path, dest_bkp)
    except OSError as exc:  # Python >2.5
        if exc.errno == 17:
            pass
        else:
            raise
            
    print "passaro Data backup: " + session_name
    sh.move(session_path, store_dest_path)

print "done"

/mnt/cube/earneodo/bci_zf/ss_data/z017/day-2016-07-07/experiment.raw.kwd
File /usr/local/experiment/raw_data/z017/2016-07-07_09-52-29_225/experiment1_100.raw.kwd
File /usr/local/experiment/raw_data/z017/2016-07-07_14-37-35_225/experiment1_100.raw.kwd
cube backup: 2016-07-07_14-37-35_225
passaro Data backup: 2016-07-07_14-37-35_225
cube backup: 2016-07-07_09-52-29_225
passaro Data backup: 2016-07-07_09-52-29_225
done


In [195]:
rolling_method = 'pandas'
rolling_method is 'pandas'

True